In [2]:
import os
import json
import numpy as np
from collections import defaultdict, Counter

In [3]:
import torch
object_vocab = torch.load('/root/data_alfred/json_feat_2.1.0/objects_20200522.vocab')

In [6]:
sorted(object_vocab['object_type'].index2word(list(range(111))))

['<<pad>>',
 'None',
 'alarmclock',
 'apple',
 'armchair',
 'baseballbat',
 'basketball',
 'bathtub',
 'bed',
 'blinds',
 'book',
 'boots',
 'bowl',
 'box',
 'bread',
 'butterknife',
 'cabinet',
 'candle',
 'cart',
 'cd',
 'cellphone',
 'chair',
 'cloth',
 'coffeemachine',
 'coffeetable',
 'countertop',
 'creditcard',
 'cup',
 'curtains',
 'desk',
 'desklamp',
 'diningtable',
 'dishsponge',
 'drawer',
 'dresser',
 'egg',
 'faucet',
 'floorlamp',
 'footstool',
 'fork',
 'fridge',
 'garbagecan',
 'glassbottle',
 'handtowel',
 'handtowelholder',
 'houseplant',
 'kettle',
 'keychain',
 'knife',
 'ladle',
 'lamp',
 'laptop',
 'laundryhamper',
 'laundryhamperlid',
 'lettuce',
 'lightswitch',
 'microwave',
 'mirror',
 'mug',
 'newspaper',
 'none',
 'ottoman',
 'painting',
 'pan',
 'papertowelroll',
 'pen',
 'pencil',
 'peppershaker',
 'pillow',
 'plate',
 'plunger',
 'poster',
 'pot',
 'potato',
 'remotecontrol',
 'safe',
 'saltshaker',
 'scrubbrush',
 'shelf',
 'showercurtain',
 'showerdoor'

## LOAD RESULTS

In [8]:
# load model results
mod21_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_50_enc_max_pool_aux_loss/'
mod22_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_43_obj_instance_enc_max_pool_aux_loss/'
mod22_wt11_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce/'
mod22_wt12_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2/'
mod23_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_biattn_axu_loss/'
mod23_wt12_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_biattn_dec_axu_loss_weighted_bce_1to2/'

In [9]:
# select by best BLEU score

seen_21_epoch = 11
seen_22_epoch = 11
seen_22_wt11_epoch = 29
seen_22_wt12_epoch = 32
seen_23_epoch = 25


unseen_21_epoch = 6
unseen_22_epoch = 12
unseen_22_wt11_epoch = 29
seen_22_wt12_epoch = 32
unseen_23_epoch = 25

In [10]:
mod22_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_43_obj_instance_enc_max_pool_aux_loss/'

with open(mod22_dout + 'train_sanity_argmax.debug_epoch_11.preds.json', 'r') as f:
    mod22_train_sanity = json.load(f)
    
with open(mod22_dout + 'valid_seen_argmax.debug_epoch_11.preds.json', 'r') as f:
    mod22_valid_seen = json.load(f)
    
with open(mod22_dout + 'valid_unseen_argmax.debug_epoch_12.preds.json', 'r') as f:
    mod22_valid_unseen = json.load(f)

In [11]:
mod22_wt11_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce/'

with open(mod22_wt11_dout + 'train_sanity_argmax.debug_epoch_29.preds.json', 'r') as f:
    mod22_wt11_train_sanity = json.load(f)
    
with open(mod22_wt11_dout + 'valid_seen_argmax.debug_epoch_29.preds.json', 'r') as f:
    mod22_wt11_valid_seen = json.load(f)
    
with open(mod22_wt11_dout + 'valid_unseen_argmax.debug_epoch_29.preds.json', 'r') as f:
    mod22_wt11_valid_unseen = json.load(f)

In [12]:
mod22_wt12_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2/'

with open(mod22_wt12_dout + 'train_sanity_argmax.debug_epoch_32.preds.json', 'r') as f:
    mod22_wt12_train_sanity = json.load(f)
    
with open(mod22_wt12_dout + 'valid_seen_argmax.debug_epoch_32.preds.json', 'r') as f:
    mod22_wt12_valid_seen = json.load(f)
    
with open(mod22_wt12_dout + 'valid_unseen_argmax.debug_epoch_32.preds.json', 'r') as f:
    mod22_wt12_valid_unseen = json.load(f)

In [13]:
mod23_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_biattn_axu_loss/'

with open(mod23_dout + 'train_sanity_argmax.debug_epoch_25.preds.json', 'r') as f:
    mod23_train_sanity = json.load(f)
    
with open(mod23_dout + 'valid_seen_argmax.debug_epoch_25.preds.json', 'r') as f:
    mod23_valid_seen = json.load(f)
    
with open(mod23_dout + 'valid_unseen_argmax.debug_epoch_25.preds.json', 'r') as f:
    mod23_valid_unseen = json.load(f)

In [14]:
mod23_wt12_dout = '/root/data/home/hoyeung/blob_alfred_data/exp_all/model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_biattn_dec_axu_loss_weighted_bce_1to2/'

with open(mod23_wt12_dout + 'train_sanity.debug_epoch_25.preds.json', 'r') as f:
    mod23_wt12_train_sanity = json.load(f)
    
with open(mod23_wt12_dout + 'valid_seen.debug_epoch_25.preds.json', 'r') as f:
    mod23_wt12_valid_seen = json.load(f)
    
with open(mod23_wt12_dout + 'valid_unseen.debug_epoch_25.preds.json', 'r') as f:
    mod23_wt12_valid_unseen = json.load(f)

## RUN THROUGH DATA

In [15]:
def classify_type_preds(pred, gt, obj_tokens, thresh=0.5):
    '''
    Compute type version for metrics.
    pred : shape (num subgoal, num objects in the task)
    gt   : shape (num subgoal, num objects in the task)
    obj_tokens : shape (num subgoal, num objects in the task)
    '''
    pred_bool = (pred > 0.5)

    obj_types = list(set(obj_tokens[0]))
    obj_type_pred = []
    obj_type_gt = []
    num_subgoals = obj_tokens.shape[0]

    for obj_type in obj_types:
        for i in range(num_subgoals):
            typ_positions = np.squeeze(np.argwhere(obj_tokens[i] == obj_type))
            obj_type_pred.append(np.any(pred_bool[i][typ_positions]))
            obj_type_gt.append(np.any(gt[i][typ_positions]))

    obj_type_pred = np.array(obj_type_pred)
    obj_type_gt = np.array(obj_type_gt)

    acc = obj_type_pred == obj_type_gt
    tp = obj_type_pred * obj_type_gt
    fp = obj_type_pred * (obj_type_gt == 0)
    fn = (obj_type_pred == 0) * obj_type_gt

    return acc, tp, fp, fn

In [16]:
def aggregate_predictions(res):
    
    # INSTANCE
    tot_vis_count = 0
    tot_stc_count = 0
    correct_vis_count = 0
    correct_stc_count = 0
    tp_vis_count = 0
    tp_stc_count = 0
    fp_vis_count = 0
    fp_stc_count = 0
    fn_vis_count = 0
    fn_stc_count = 0

    # TYPE
    typ_tot_vis_count = 0
    typ_tot_stc_count = 0
    typ_correct_vis_count = 0
    typ_correct_stc_count = 0
    typ_tp_vis_count = 0
    typ_tp_stc_count = 0
    typ_fp_vis_count = 0
    typ_fp_stc_count = 0
    typ_fn_vis_count = 0
    typ_fn_stc_count = 0

    all_tp_stc_obj_tokens = []
    all_tp_vis_obj_tokens = []

    all_fn_stc_obj_tokens = []
    all_fn_vis_obj_tokens = []

    for task_id in list(res.keys()):

        num_subgoals = len(res[task_id]['action_high'])-1

        # create empty arrays
        obj_token_id = np.empty((num_subgoals, len(res[task_id]['obj_token_id'][str(0)])))
        p_obj_vis = np.empty((num_subgoals, len(res[task_id]['p_obj_vis'][str(0)])))
        p_state_change = np.empty((num_subgoals, len(res[task_id]['p_state_change'][str(0)])))
        l_obj_vis = np.empty((num_subgoals, len(res[task_id]['l_obj_vis'][str(0)])))
        l_state_change = np.empty((num_subgoals, len(res[task_id]['l_state_change'][str(0)])))
        assert p_obj_vis.shape == p_state_change.shape == l_obj_vis.shape == l_state_change.shape

        tot_vis_count += num_subgoals * len(res[task_id]['p_obj_vis'][str(0)])
        tot_stc_count += num_subgoals * len(res[task_id]['p_state_change'][str(0)])

        # fill empty arrays
        for i in range(num_subgoals):
            obj_token_id[i] = res[task_id]['obj_token_id'][str(i)] 
            p_obj_vis[i] = res[task_id]['p_obj_vis'][str(i)] 
            p_state_change[i] = res[task_id]['p_state_change'][str(i)]
            l_obj_vis[i] = res[task_id]['l_obj_vis'][str(i)]
            l_state_change[i] = res[task_id]['l_state_change'][str(i)]
        # apply threshold 0.5
        p_obj_vis = (p_obj_vis > 0.5).astype(np.float32)
        p_state_change = (p_state_change > 0.5).astype(np.float32)

        # INSTANCE -----------------    

        # correct
        correct_vis = (l_obj_vis == p_obj_vis).astype(np.float32)
        correct_stc = (l_state_change == p_state_change).astype(np.float32)
        correct_vis_count += np.sum(correct_vis)
        correct_stc_count += np.sum(correct_stc)

        # TP
        tp_stc = ((l_state_change == 1.0) & (p_state_change == 1.0)).astype(np.float32)
        tp_vis = ((l_obj_vis == 1.0) & (p_obj_vis == 1.0)).astype(np.float32)
        tp_stc_count += np.sum(tp_stc)
        tp_vis_count += np.sum(tp_vis) 

        # FP
        fp_stc = ((l_state_change == 0.0) & (p_state_change == 1.0)).astype(np.float32)
        fp_vis = ((l_obj_vis == 0.0) & (p_obj_vis == 1.0)).astype(np.float32)
        fp_stc_count += np.sum(fp_stc)
        fp_vis_count += np.sum(fp_vis) 

        # FN
        fn_stc = ((l_state_change == 1.0) & (p_state_change == 0.0)).astype(np.float32)
        fn_vis = ((l_obj_vis == 1.0) & (p_obj_vis == 0.0)).astype(np.float32)
        fn_stc_count += np.sum(fn_stc)
        fn_vis_count += np.sum(fn_vis) 

        # TYPE ----------------- 
        obj_types = list(set(obj_token_id[0]))
        typ_correct_vis, typ_tp_vis, typ_fp_vis, typ_fn_vis = classify_type_preds(p_obj_vis, l_obj_vis, obj_token_id)
        typ_correct_stc, typ_tp_stc, typ_fp_stc, typ_fn_stc = classify_type_preds(p_state_change, l_state_change, obj_token_id)
        typ_tot_vis_count += typ_correct_vis.shape[0]
        typ_tot_stc_count += typ_correct_stc.shape[0]
        typ_correct_vis_count += np.sum(typ_correct_vis)
        typ_correct_stc_count += np.sum(typ_correct_stc)
        typ_tp_stc_count += np.sum(typ_tp_stc)
        typ_tp_vis_count += np.sum(typ_tp_vis) 
        typ_fp_stc_count += np.sum(typ_fp_stc)
        typ_fp_vis_count += np.sum(typ_fp_vis) 
        typ_fn_stc_count += np.sum(typ_fn_stc)
        typ_fn_vis_count += np.sum(typ_fn_vis) 

        # -----------------
        # find TP indices
        tp_stc_pos = np.nonzero(tp_stc)
        tp_vis_pos = np.nonzero(tp_vis)
        
        # find FN indices 
        fn_stc_pos = np.nonzero(fn_stc)
        fn_vis_pos = np.nonzero(fn_vis)
        
        # identify TP object types
        obj_token_tp_stc = obj_token_id[tp_stc_pos]
        obj_token_tp_vis = obj_token_id[tp_vis_pos]
        all_tp_stc_obj_tokens.extend(obj_token_tp_stc.tolist())
        all_tp_vis_obj_tokens.extend(obj_token_tp_vis.tolist())
    
        # identify FN object types
        obj_token_fn_stc = obj_token_id[fn_stc_pos]
        obj_token_fn_vis = obj_token_id[fn_vis_pos]
        all_fn_stc_obj_tokens.extend(obj_token_fn_stc.tolist())
        all_fn_vis_obj_tokens.extend(obj_token_fn_vis.tolist())    
    
    aggregated_res =  {
        'tot_vis_count':tot_vis_count,
        'tot_stc_count':tot_stc_count,
        'correct_vis_count':correct_vis_count,
        'correct_stc_count':correct_stc_count,
        'tp_vis_count': tp_vis_count,
        'tp_stc_count': tp_stc_count,
        'fp_vis_count': fp_vis_count,
        'fp_stc_count': fp_stc_count,
        'fn_vis_count': fn_vis_count ,
        'fn_stc_count': fn_stc_count,
        
        'typ_tot_vis_count':typ_tot_vis_count,
        'typ_tot_stc_count':typ_tot_stc_count,
        'typ_correct_vis_count':typ_correct_vis_count,
        'typ_correct_stc_count':typ_correct_stc_count,
        'typ_tp_vis_count':typ_tp_vis_count,
        'typ_tp_stc_count':typ_tp_stc_count,
        'typ_fp_vis_count':typ_fp_vis_count,
        'typ_fp_stc_count':typ_fp_stc_count,
        'typ_fn_vis_count':typ_fn_vis_count,
        'typ_fn_stc_count':typ_fn_stc_count }
    
    return aggregated_res, all_tp_stc_obj_tokens, all_tp_vis_obj_tokens, all_fn_stc_obj_tokens, all_fn_vis_obj_tokens

## ACC, RECALL , PRECISION

In [21]:
# mod 23 wt12
mod23_w12_train_sanity_res, mod23_w12_train_sanity_tp_stc_obj_tokens, mod23_w12_train_sanity_tp_vis_obj_tokens, mod23_w12_train_sanity_fn_stc_obj_tokens, mod23_w12_train_sanity_fn_vis_obj_tokens = aggregate_predictions(mod23_wt12_train_sanity)
mod23_w12_valid_seen_res, mod23_w12_valid_seen_tp_stc_obj_tokens, mod23_w12_valid_seen_tp_vis_obj_tokens, mod23_w12_valid_seen_fn_stc_obj_tokens, mod23_w12_valid_seen_fn_vis_obj_tokens = aggregate_predictions(mod23_wt12_valid_seen)
mod23_w12_valid_unseen_res, mod23_w12_valid_unseen_tp_stc_obj_tokens, mod23_w12_valid_unseen_tp_vis_obj_tokens, mod23_w12_valid_unseen_fn_stc_obj_tokens, mod23_w12_valid_unseen_fn_vis_obj_tokens = aggregate_predictions(mod23_wt12_valid_unseen)

In [46]:
res = mod23_w12_valid_unseen_res

In [47]:
# ACC STC
res['typ_correct_stc_count'] / res['typ_tot_stc_count']

0.9432985413935583

In [48]:
# ACC VIS
res['typ_correct_vis_count'] / res['typ_tot_vis_count']

0.7856562069594942

In [49]:
# RECALL VIS
res['typ_tp_vis_count'] / (res['typ_tp_vis_count'] + res['typ_fn_vis_count'])

0.3635346756152125

In [50]:
# PRECISION VIS
res['typ_tp_vis_count'] / (res['typ_tp_vis_count'] + res['typ_fp_vis_count'])

0.4335556695741406

In [51]:
# RECALL STC
res['typ_tp_stc_count'] / (res['typ_tp_stc_count'] + res['typ_fn_stc_count'])

0.8577661431064573

In [52]:
# PRECISION STC
res['typ_tp_stc_count'] / (res['typ_tp_stc_count'] + res['typ_fp_stc_count'])

0.23200377625678545

## WHICH OBJECTS DO WE GET RIGHT?

In [ ]:
mod22_train_sanity
mod22_valid_seen
mod22_valid_unseen

mod22_wt11_train_sanity
mod22_wt11_valid_seen
mod22_wt11_valid_unseen

mod22_wt12_train_sanity
mod22_wt12_valid_seen
mod22_wt12_valid_unseen

mod23_train_sanity
mod23_valid_seen
mod23_valid_unseen

In [54]:
def print_most_common(obj_tokens):
    ranked = [(object_vocab['object_type'].index2word(int(obj)),count) for (obj, count) in Counter(obj_tokens).most_common(20)]
    for item in ranked:
        print(item)

### mod 22

In [196]:
# mod 22
mod22_train_sanity_res, mod22_train_sanity_tp_stc_obj_tokens, mod22_train_sanity_tp_vis_obj_tokens, mod22_train_sanity_fn_stc_obj_tokens, mod22_train_sanity_fn_vis_obj_tokens = aggregate_predictions(mod22_train_sanity)
mod22_valid_seen_res, mod22_valid_seen_tp_stc_obj_tokens, mod22_valid_seen_tp_vis_obj_tokens, mod22_valid_seen_fn_stc_obj_tokens, mod22_valid_seen_fn_vis_obj_tokens = aggregate_predictions(mod22_valid_seen)
mod22_valid_unseen_res, mod22_valid_unseen_tp_stc_obj_tokens, mod22_valid_unseen_tp_vis_obj_tokens, mod22_valid_unseen_fn_stc_obj_tokens, mod22_valid_unseen_fn_vis_obj_tokens = aggregate_predictions(mod22_valid_unseen)

In [201]:
# mod 22 STAGE CHANGE TP
print('Train Sanity')
print_most_common(mod22_train_sanity_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_valid_seen_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_valid_unseen_tp_stc_obj_tokens)

Train Sanity
('tomato', 19)
('apple', 18)
('butterknife', 8)
('remotecontrol', 4)
('spoon', 3)
('spatula', 3)
('creditcard', 3)
('cellphone', 3)
('keychain', 2)
('fork', 1)
--------------------------------
Valid Seen
('potato', 26)
('lettuce', 8)
('apple', 8)
('tomato', 7)
('remotecontrol', 7)
('butterknife', 3)
('dishsponge', 2)
('newspaper', 2)
('spoon', 2)
('fork', 1)
--------------------------------
Valid Unseen
('fork', 12)
('butterknife', 8)
('potato', 8)
('keychain', 4)
('knife', 4)


In [205]:
# mod 22 STAGE CHANGE FN
print('Train Sanity')
print_most_common(mod22_train_sanity_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_valid_seen_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_valid_unseen_fn_stc_obj_tokens)

Train Sanity
('potato', 132)
('tomato', 108)
('egg', 101)
('apple', 94)
('lettuce', 80)
('bread', 76)
('knife', 68)
('mug', 48)
('butterknife', 46)
('cup', 36)
('cloth', 35)
('soapbar', 30)
('plate', 27)
('pan', 27)
('spoon', 26)
('pencil', 24)
('pot', 24)
('cellphone', 21)
('bowl', 20)
('spatula', 20)
--------------------------------
Valid Seen
('tomato', 181)
('apple', 119)
('egg', 100)
('lettuce', 88)
('potato', 75)
('knife', 66)
('butterknife', 54)
('bread', 45)
('mug', 41)
('plate', 37)
('soapbar', 34)
('cup', 28)
('spraybottle', 28)
('ladle', 26)
('pan', 25)
('remotecontrol', 22)
('microwave', 21)
('creditcard', 17)
('bowl', 17)
('toiletpaper', 17)
--------------------------------
Valid Unseen
('lettuce', 227)
('egg', 219)
('apple', 108)
('tomato', 86)
('bread', 79)
('mug', 64)
('butterknife', 59)
('potato', 51)
('cup', 48)
('soapbar', 36)
('knife', 35)
('pan', 31)
('floorlamp', 30)
('bowl', 26)
('keychain', 25)
('spoon', 24)
('desklamp', 23)
('pencil', 21)
('peppershaker', 18)
(

In [215]:
# mod 22 VIS TP
print('Train Sanity')
print_most_common(mod22_train_sanity_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_valid_seen_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_valid_unseen_tp_vis_obj_tokens)

Train Sanity
('sink', 429)
('drawer', 283)
('tomato', 267)
('countertop', 212)
('potato', 199)
('faucet', 179)
('lettuce', 165)
('mug', 159)
('bread', 147)
('knife', 132)
('butterknife', 122)
('spatula', 121)
('apple', 118)
('soapbottle', 117)
('fork', 114)
('cup', 114)
('bowl', 110)
('spoon', 108)
('fridge', 101)
('soapbar', 90)
--------------------------------
Valid Seen
('sink', 482)
('tomato', 277)
('apple', 219)
('drawer', 209)
('countertop', 208)
('faucet', 168)
('lettuce', 166)
('bread', 159)
('knife', 152)
('butterknife', 143)
('potato', 131)
('spatula', 125)
('cup', 107)
('fridge', 106)
('fork', 104)
('soapbottle', 100)
('mug', 99)
('bowl', 94)
('dishsponge', 92)
('spoon', 87)
--------------------------------
Valid Unseen
('sink', 506)
('countertop', 390)
('faucet', 302)
('tomato', 167)
('cabinet', 162)
('bread', 160)
('butterknife', 159)
('apple', 135)
('potato', 111)
('knife', 103)
('egg', 101)
('cellphone', 99)
('lettuce', 99)
('soapbar', 97)
('desk', 96)
('mirror', 85)
('m

In [216]:
# mod 22 VIS FN
print('Train Sanity')
print_most_common(mod22_train_sanity_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_valid_seen_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_valid_unseen_fn_vis_obj_tokens)

Train Sanity
('drawer', 1197)
('cabinet', 1145)
('countertop', 515)
('stoveburner', 323)
('sink', 253)
('chair', 241)
('stoveknob', 217)
('potato', 216)
('bread', 189)
('diningtable', 186)
('knife', 182)
('shelf', 170)
('butterknife', 168)
('fork', 165)
('tomato', 160)
('pot', 153)
('mug', 152)
('spatula', 152)
('faucet', 149)
('fridge', 145)
--------------------------------
Valid Seen
('cabinet', 1423)
('drawer', 1019)
('countertop', 536)
('sink', 302)
('stoveburner', 268)
('stoveknob', 241)
('tomato', 237)
('bread', 235)
('lettuce', 197)
('knife', 184)
('faucet', 182)
('chair', 181)
('spatula', 178)
('apple', 177)
('diningtable', 164)
('fork', 157)
('butterknife', 156)
('pot', 155)
('garbagecan', 155)
('pan', 151)
--------------------------------
Valid Unseen
('drawer', 769)
('cabinet', 725)
('countertop', 517)
('lettuce', 329)
('butterknife', 270)
('statue', 269)
('shelf', 266)
('tomato', 262)
('bread', 260)
('apple', 251)
('sink', 241)
('stoveburner', 240)
('knife', 237)
('pot', 22

### mod22_wt11

In [197]:
# mmod22_wt11
mod22_wt11_train_sanity_res, mod22_wt11_train_sanity_tp_stc_obj_tokens, mod22_wt11_train_sanity_tp_vis_obj_tokens, mod22_wt11_train_sanity_fn_stc_obj_tokens, mod22_wt11_train_sanity_fn_vis_obj_tokens = aggregate_predictions(mod22_wt11_train_sanity)
mod22_wt11_valid_seen_res, mod22_wt11_valid_seen_tp_stc_obj_tokens, mod22_wt11_valid_seen_tp_vis_obj_tokens, mod22_wt11_valid_seen_fn_stc_obj_tokens, mod22_wt11_valid_seen_fn_vis_obj_tokens = aggregate_predictions(mod22_wt11_valid_seen)
mod22_wt11_valid_unseen_res, mod22_wt11_valid_unseen_tp_stc_obj_tokens, mod22_wt11_valid_unseen_tp_vis_obj_tokens, mod22_wt11_valid_unseen_fn_stc_obj_tokens, mod22_wt11_valid_unseen_fn_vis_obj_tokens = aggregate_predictions(mod22_wt11_valid_unseen)

In [202]:
# mod22_wt11 STAGE CHANGE TP
print('Train Sanity')
print_most_common(mod22_wt11_train_sanity_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt11_valid_seen_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt11_valid_unseen_tp_stc_obj_tokens)

Train Sanity
('potato', 129)
('tomato', 118)
('apple', 98)
('egg', 84)
('bread', 76)
('knife', 63)
('lettuce', 61)
('butterknife', 53)
('mug', 47)
('cup', 36)
('cloth', 34)
('spoon', 27)
('soapbar', 27)
('pan', 26)
('cellphone', 24)
('plate', 23)
('pencil', 22)
('pot', 20)
('spatula', 20)
('bowl', 19)
--------------------------------
Valid Seen
('tomato', 172)
('apple', 124)
('potato', 90)
('egg', 85)
('lettuce', 81)
('knife', 66)
('butterknife', 57)
('bread', 45)
('mug', 40)
('soapbar', 33)
('plate', 31)
('cup', 28)
('spraybottle', 27)
('remotecontrol', 27)
('ladle', 25)
('pan', 25)
('creditcard', 17)
('bowl', 17)
('microwave', 16)
('desklamp', 16)
--------------------------------
Valid Unseen
('egg', 191)
('lettuce', 122)
('apple', 108)
('tomato', 86)
('bread', 78)
('butterknife', 67)
('mug', 61)
('potato', 58)
('cup', 47)
('knife', 39)
('soapbar', 35)
('pan', 31)
('floorlamp', 30)
('keychain', 26)
('spoon', 23)
('desklamp', 23)
('pencil', 21)
('fork', 21)
('bowl', 20)
('cloth', 16)


In [206]:
# mod22_wt11 STAGE CHANGE FN
print('Train Sanity')
print_most_common(mod22_wt11_train_sanity_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt11_valid_seen_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt11_valid_unseen_fn_stc_obj_tokens)

Train Sanity
('lettuce', 19)
('egg', 17)
('apple', 14)
('cabinet', 13)
('tomato', 9)
('knife', 5)
('pot', 4)
('plate', 4)
('toiletpaper', 3)
('soapbar', 3)
('spatula', 3)
('potato', 3)
('ladle', 2)
('microwave', 2)
('soapbottle', 2)
('spoon', 2)
('keychain', 2)
('pencil', 2)
('plunger', 2)
('bowl', 1)
--------------------------------
Valid Seen
('tomato', 16)
('lettuce', 15)
('egg', 15)
('potato', 11)
('plate', 6)
('winebottle', 6)
('microwave', 5)
('plunger', 4)
('sink', 3)
('apple', 3)
('spatula', 3)
('fork', 3)
('soapbottle', 2)
('vase', 2)
('toiletpaper', 2)
('remotecontrol', 2)
('cabinet', 2)
('spraybottle', 1)
('mug', 1)
('soapbar', 1)
--------------------------------
Valid Unseen
('lettuce', 105)
('egg', 28)
('cabinet', 14)
('baseballbat', 9)
('saltshaker', 7)
('plunger', 6)
('bowl', 6)
('peppershaker', 5)
('mug', 3)
('plate', 3)
('keychain', 3)
('coffeemachine', 3)
('newspaper', 2)
('cloth', 2)
('tissuebox', 2)
('vase', 2)
('spatula', 1)
('cup', 1)
('bread', 1)
('fork', 1)


In [214]:
# mod22_wt11 VIS TP
print('Train Sanity')
print_most_common(mod22_wt11_train_sanity_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt11_valid_seen_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt11_valid_unseen_tp_vis_obj_tokens)

Train Sanity
('drawer', 1227)
('cabinet', 789)
('countertop', 694)
('sink', 625)
('tomato', 402)
('potato', 373)
('bread', 318)
('stoveburner', 311)
('faucet', 293)
('knife', 293)
('mug', 289)
('butterknife', 264)
('lettuce', 259)
('chair', 247)
('spatula', 238)
('fork', 234)
('apple', 232)
('diningtable', 231)
('fridge', 224)
('spoon', 221)
--------------------------------
Valid Seen
('cabinet', 981)
('drawer', 951)
('countertop', 727)
('sink', 688)
('tomato', 489)
('apple', 370)
('bread', 361)
('lettuce', 346)
('knife', 322)
('faucet', 301)
('butterknife', 280)
('spatula', 271)
('potato', 254)
('stoveburner', 242)
('fork', 226)
('diningtable', 220)
('fridge', 214)
('mug', 190)
('soapbottle', 188)
('pan', 175)
--------------------------------
Valid Unseen
('countertop', 902)
('sink', 731)
('drawer', 641)
('cabinet', 583)
('lettuce', 391)
('faucet', 383)
('butterknife', 374)
('bread', 372)
('tomato', 360)
('apple', 337)
('knife', 314)
('potato', 267)
('spatula', 241)
('mug', 235)
('cel

In [217]:
# mod22_wt11 VIS FN
print('Train Sanity')
print_most_common(mod22_wt11_train_sanity_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt11_valid_seen_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt11_valid_unseen_fn_vis_obj_tokens)

Train Sanity
('cabinet', 391)
('drawer', 253)
('window', 76)
('stoveknob', 71)
('chair', 70)
('sink', 57)
('shelf', 51)
('stoveburner', 49)
('soapbottle', 47)
('fork', 45)
('garbagecan', 42)
('potato', 42)
('faucet', 35)
('spatula', 35)
('countertop', 33)
('peppershaker', 31)
('microwave', 28)
('dishsponge', 28)
('pan', 26)
('sidetable', 26)
--------------------------------
Valid Seen
('cabinet', 494)
('drawer', 277)
('sink', 96)
('stoveknob', 72)
('window', 67)
('chair', 66)
('garbagecan', 57)
('stoveburner', 54)
('shelf', 50)
('faucet', 49)
('handtowel', 40)
('soapbottle', 36)
('fork', 35)
('bread', 33)
('microwave', 32)
('spatula', 32)
('spoon', 31)
('toiletpaper', 29)
('apple', 26)
('tomato', 25)
--------------------------------
Valid Unseen
('cabinet', 304)
('drawer', 208)
('shelf', 150)
('statue', 134)
('stoveburner', 131)
('scrubbrush', 75)
('tomato', 69)
('houseplant', 63)
('chair', 60)
('butterknife', 55)
('stoveknob', 55)
('garbagecan', 55)
('cellphone', 52)
('apple', 49)
('b

### mod22_wt12

In [198]:
# mmod22_wt12
mod22_wt12_train_sanity_res, mod22_wt12_train_sanity_tp_stc_obj_tokens, mod22_wt12_train_sanity_tp_vis_obj_tokens, mod22_wt12_train_sanity_fn_stc_obj_tokens, mod22_wt12_train_sanity_fn_vis_obj_tokens = aggregate_predictions(mod22_wt12_train_sanity)
mod22_wt12_valid_seen_res, mod22_wt12_valid_seen_tp_stc_obj_tokens, mod22_wt12_valid_seen_tp_vis_obj_tokens, mod22_wt12_valid_seen_fn_stc_obj_tokens, mod22_wt12_valid_seen_fn_vis_obj_tokens = aggregate_predictions(mod22_wt12_valid_seen)
mod22_wt12_valid_unseen_res, mod22_wt12_valid_unseen_tp_stc_obj_tokens, mod22_wt12_valid_unseen_tp_vis_obj_tokens, mod22_wt12_valid_unseen_fn_stc_obj_tokens, mod22_wt12_valid_unseen_fn_vis_obj_tokens = aggregate_predictions(mod22_wt12_valid_unseen)

In [203]:
# mod22_wt12 STAGE CHANGE TP
print('Train Sanity')
print_most_common(mod22_wt12_train_sanity_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt12_valid_seen_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt12_valid_unseen_tp_stc_obj_tokens)

Train Sanity
('potato', 111)
('tomato', 99)
('apple', 88)
('egg', 68)
('knife', 63)
('bread', 56)
('butterknife', 52)
('mug', 46)
('lettuce', 44)
('cloth', 34)
('cup', 33)
('spoon', 27)
('soapbar', 27)
('pan', 26)
('cellphone', 24)
('plate', 22)
('pot', 22)
('keychain', 20)
('pencil', 19)
('spatula', 19)
--------------------------------
Valid Seen
('tomato', 158)
('apple', 116)
('potato', 76)
('knife', 65)
('egg', 61)
('lettuce', 58)
('butterknife', 54)
('mug', 36)
('bread', 34)
('soapbar', 33)
('cup', 28)
('plate', 28)
('spraybottle', 27)
('remotecontrol', 27)
('pan', 23)
('ladle', 22)
('desklamp', 16)
('bowl', 15)
('keychain', 15)
('creditcard', 14)
--------------------------------
Valid Unseen
('egg', 160)
('apple', 108)
('lettuce', 87)
('tomato', 70)
('butterknife', 67)
('mug', 58)
('potato', 57)
('bread', 46)
('cup', 41)
('knife', 37)
('soapbar', 35)
('floorlamp', 30)
('pan', 27)
('keychain', 26)
('desklamp', 23)
('spoon', 22)
('pencil', 21)
('bowl', 20)
('fork', 20)
('plate', 15)

In [207]:
# mod22_wt12 STAGE CHANGE FN
print('Train Sanity')
print_most_common(mod22_wt12_train_sanity_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt12_valid_seen_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt12_valid_unseen_fn_stc_obj_tokens)

Train Sanity
('lettuce', 36)
('egg', 33)
('tomato', 28)
('apple', 24)
('potato', 21)
('bread', 20)
('cabinet', 13)
('microwave', 6)
('winebottle', 5)
('plate', 5)
('pencil', 5)
('knife', 5)
('spatula', 4)
('bowl', 3)
('toiletpaper', 3)
('cup', 3)
('soapbar', 3)
('pen', 2)
('butterknife', 2)
('mug', 2)
--------------------------------
Valid Seen
('egg', 39)
('lettuce', 38)
('tomato', 30)
('potato', 25)
('apple', 11)
('bread', 11)
('microwave', 10)
('plate', 9)
('winebottle', 8)
('toiletpaper', 6)
('mug', 5)
('spatula', 5)
('fork', 5)
('ladle', 4)
('sink', 4)
('plunger', 4)
('spoon', 3)
('soapbottle', 3)
('butterknife', 3)
('cloth', 3)
--------------------------------
Valid Unseen
('lettuce', 140)
('egg', 59)
('bread', 33)
('tomato', 16)
('cabinet', 14)
('saltshaker', 11)
('peppershaker', 8)
('cup', 7)
('mug', 6)
('plunger', 6)
('bowl', 6)
('cloth', 5)
('pan', 4)
('watch', 4)
('plate', 3)
('keychain', 3)
('spatula', 3)
('baseballbat', 3)
('coffeemachine', 3)
('vase', 3)


In [212]:
# mod22_wt12 VIS TP
print('Train Sanity')
print_most_common(mod22_wt12_train_sanity_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt12_valid_seen_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt12_valid_unseen_tp_vis_obj_tokens)

Train Sanity
('drawer', 1122)
('countertop', 651)
('cabinet', 622)
('sink', 619)
('tomato', 388)
('potato', 375)
('stoveburner', 322)
('bread', 300)
('faucet', 287)
('mug', 286)
('knife', 284)
('butterknife', 262)
('chair', 261)
('lettuce', 256)
('fork', 236)
('fridge', 232)
('apple', 230)
('spatula', 225)
('spoon', 222)
('diningtable', 214)
--------------------------------
Valid Seen
('drawer', 892)
('cabinet', 872)
('sink', 707)
('countertop', 676)
('tomato', 473)
('apple', 372)
('bread', 340)
('lettuce', 337)
('knife', 308)
('faucet', 295)
('butterknife', 272)
('stoveburner', 262)
('spatula', 261)
('potato', 247)
('fork', 226)
('fridge', 216)
('diningtable', 212)
('chair', 197)
('soapbottle', 190)
('mug', 189)
--------------------------------
Valid Unseen
('countertop', 783)
('sink', 708)
('drawer', 493)
('cabinet', 486)
('faucet', 371)
('lettuce', 360)
('tomato', 342)
('apple', 341)
('butterknife', 322)
('bread', 293)
('knife', 252)
('potato', 247)
('mug', 234)
('stoveburner', 225)

In [213]:
# mod22_wt12 VIS FN
print('Train Sanity')
print_most_common(mod22_wt12_train_sanity_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod22_wt12_valid_seen_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod22_wt12_valid_unseen_fn_vis_obj_tokens)

Train Sanity
('cabinet', 558)
('drawer', 358)
('window', 80)
('countertop', 76)
('shelf', 65)
('sink', 63)
('chair', 56)
('spatula', 48)
('stoveknob', 45)
('soapbottle', 44)
('fork', 43)
('faucet', 41)
('potato', 40)
('tomato', 39)
('stoveburner', 38)
('bread', 36)
('garbagecan', 34)
('toiletpaper', 31)
('knife', 30)
('diningtable', 30)
--------------------------------
Valid Seen
('cabinet', 603)
('drawer', 336)
('sink', 77)
('stoveknob', 71)
('countertop', 68)
('window', 57)
('faucet', 55)
('garbagecan', 55)
('bread', 54)
('handtowel', 45)
('chair', 42)
('spatula', 42)
('tomato', 41)
('toiletpaper', 35)
('fork', 35)
('mirror', 34)
('soapbottle', 34)
('stoveburner', 34)
('plate', 33)
('shelf', 33)
--------------------------------
Valid Unseen
('cabinet', 401)
('drawer', 356)
('statue', 211)
('bread', 127)
('shelf', 125)
('countertop', 124)
('cellphone', 119)
('butterknife', 107)
('glassbottle', 91)
('knife', 88)
('houseplant', 87)
('tomato', 87)
('scrubbrush', 78)
('stoveburner', 75)
(

### mod 23

In [199]:
# mod 23
mod23_train_sanity_res, mod23_train_sanity_tp_stc_obj_tokens, mod23_train_sanity_tp_vis_obj_tokens, mod23_train_sanity_fn_stc_obj_tokens, mod23_train_sanity_fn_vis_obj_tokens = aggregate_predictions(mod23_train_sanity)
mod23_valid_seen_res, mod23_valid_seen_tp_stc_obj_tokens, mod23_valid_seen_tp_vis_obj_tokens, mod23_valid_seen_fn_stc_obj_tokens, mod23_valid_seen_fn_vis_obj_tokens = aggregate_predictions(mod23_valid_seen)
mod23_valid_unseen_res, mod23_valid_unseen_tp_stc_obj_tokens, mod23_valid_unseen_tp_vis_obj_tokens, mod23_valid_unseen_fn_stc_obj_tokens, mod23_valid_unseen_fn_vis_obj_tokens = aggregate_predictions(mod23_valid_unseen)

In [204]:
# mod 23 STAGE CHANGE TP
print('Train Sanity')
print_most_common(mod23_train_sanity_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_valid_seen_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_valid_unseen_tp_stc_obj_tokens)

Train Sanity
('tomato', 18)
('apple', 16)
('butterknife', 11)
('spoon', 6)
('cellphone', 6)
('egg', 3)
('pencil', 3)
('spatula', 3)
('creditcard', 3)
('remotecontrol', 2)
('cloth', 2)
('knife', 2)
('pen', 2)
('lettuce', 2)
('handtowel', 2)
('keychain', 2)
('watch', 1)
('bread', 1)
('soapbar', 1)
('alarmclock', 1)
--------------------------------
Valid Seen
('potato', 28)
('lettuce', 9)
('apple', 7)
('remotecontrol', 6)
('egg', 5)
('tomato', 4)
('newspaper', 4)
('butterknife', 4)
('pen', 2)
('knife', 2)
('soapbottle', 1)
('bread', 1)
('spoon', 1)
('fridge', 1)
('plate', 1)
('soapbar', 1)
('dishsponge', 1)
('toiletpaper', 1)
('handtowel', 1)
--------------------------------
Valid Unseen
('egg', 13)
('potato', 11)
('butterknife', 10)
('keychain', 4)
('spoon', 2)
('soapbar', 2)
('toiletpaper', 2)
('knife', 1)
('cloth', 1)


In [209]:
# mod 23 STAGE CHANGE FN
print('Train Sanity')
print_most_common(mod23_train_sanity_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_valid_seen_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_valid_unseen_fn_stc_obj_tokens)

Train Sanity
('potato', 131)
('tomato', 109)
('egg', 98)
('apple', 96)
('lettuce', 78)
('bread', 75)
('knife', 66)
('mug', 48)
('butterknife', 43)
('cup', 35)
('cloth', 33)
('soapbar', 29)
('plate', 27)
('pan', 27)
('pot', 24)
('spoon', 23)
('pencil', 21)
('bowl', 20)
('spatula', 20)
('keychain', 19)
--------------------------------
Valid Seen
('tomato', 184)
('apple', 120)
('egg', 95)
('lettuce', 87)
('potato', 73)
('knife', 64)
('butterknife', 53)
('bread', 44)
('mug', 41)
('plate', 36)
('soapbar', 33)
('cup', 28)
('spraybottle', 28)
('ladle', 26)
('pan', 25)
('remotecontrol', 23)
('microwave', 21)
('creditcard', 17)
('bowl', 17)
('desklamp', 16)
--------------------------------
Valid Unseen
('lettuce', 227)
('egg', 206)
('apple', 108)
('tomato', 86)
('bread', 79)
('mug', 64)
('butterknife', 57)
('cup', 48)
('potato', 48)
('knife', 38)
('soapbar', 34)
('pan', 31)
('floorlamp', 30)
('bowl', 26)
('keychain', 25)
('desklamp', 23)
('spoon', 22)
('fork', 22)
('pencil', 21)
('peppershaker'

In [211]:
# mod 23 VIS TP
print('Train Sanity')
print_most_common(mod23_train_sanity_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_valid_seen_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_valid_unseen_tp_vis_obj_tokens)

Train Sanity
('knife', 83)
('butterknife', 72)
('countertop', 65)
('desk', 52)
('potato', 52)
('mug', 44)
('apple', 41)
('tomato', 40)
('lettuce', 35)
('cloth', 33)
('cup', 31)
('soapbar', 30)
('pot', 22)
('creditcard', 21)
('handtowel', 21)
('cellphone', 21)
('pencil', 20)
('pan', 20)
('bread', 17)
('bowl', 16)
--------------------------------
Valid Seen
('knife', 83)
('tomato', 81)
('butterknife', 76)
('apple', 64)
('countertop', 46)
('soapbar', 41)
('desk', 34)
('mug', 32)
('lettuce', 31)
('spraybottle', 28)
('potato', 25)
('remotecontrol', 24)
('pan', 20)
('creditcard', 19)
('plate', 18)
('bowl', 17)
('cup', 17)
('faucet', 16)
('cd', 15)
('microwave', 15)
--------------------------------
Valid Unseen
('butterknife', 77)
('apple', 71)
('mug', 67)
('tomato', 49)
('knife', 49)
('desk', 48)
('countertop', 43)
('soapbar', 42)
('lettuce', 40)
('egg', 36)
('desklamp', 30)
('toiletpaper', 30)
('cup', 29)
('potato', 28)
('bowl', 27)
('bread', 26)
('pan', 25)
('cloth', 22)
('keychain', 18)
(

In [210]:
# mod 23 VIS FN
print('Train Sanity')
print_most_common(mod23_train_sanity_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_valid_seen_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_valid_unseen_fn_vis_obj_tokens)

Train Sanity
('drawer', 1480)
('cabinet', 1180)
('sink', 671)
('countertop', 662)
('tomato', 387)
('potato', 363)
('stoveburner', 360)
('bread', 319)
('chair', 317)
('faucet', 312)
('fork', 271)
('mug', 267)
('spatula', 263)
('fridge', 246)
('lettuce', 245)
('diningtable', 243)
('knife', 231)
('soapbottle', 231)
('spoon', 225)
('butterknife', 218)
--------------------------------
Valid Seen
('cabinet', 1475)
('drawer', 1228)
('sink', 777)
('countertop', 698)
('tomato', 433)
('bread', 381)
('faucet', 334)
('apple', 332)
('lettuce', 332)
('spatula', 299)
('stoveburner', 296)
('knife', 253)
('potato', 251)
('fork', 247)
('stoveknob', 241)
('chair', 239)
('fridge', 235)
('diningtable', 230)
('butterknife', 223)
('soapbottle', 217)
--------------------------------
Valid Unseen
('cabinet', 887)
('countertop', 864)
('drawer', 849)
('sink', 735)
('faucet', 418)
('bread', 394)
('lettuce', 388)
('tomato', 380)
('butterknife', 352)
('apple', 315)
('stoveburner', 300)
('knife', 291)
('statue', 285

### mod23_wt12

In [21]:
# mod 23 wt12
mod23_w12_train_sanity_res, mod23_w12_train_sanity_tp_stc_obj_tokens, mod23_w12_train_sanity_tp_vis_obj_tokens, mod23_w12_train_sanity_fn_stc_obj_tokens, mod23_w12_train_sanity_fn_vis_obj_tokens = aggregate_predictions(mod23_wt12_train_sanity)
mod23_w12_valid_seen_res, mod23_w12_valid_seen_tp_stc_obj_tokens, mod23_w12_valid_seen_tp_vis_obj_tokens, mod23_w12_valid_seen_fn_stc_obj_tokens, mod23_w12_valid_seen_fn_vis_obj_tokens = aggregate_predictions(mod23_wt12_valid_seen)
mod23_w12_valid_unseen_res, mod23_w12_valid_unseen_tp_stc_obj_tokens, mod23_w12_valid_unseen_tp_vis_obj_tokens, mod23_w12_valid_unseen_fn_stc_obj_tokens, mod23_w12_valid_unseen_fn_vis_obj_tokens = aggregate_predictions(mod23_wt12_valid_unseen)

In [55]:
# mod 23 wt12 STAGE CHANGE TP
print('Train Sanity')
print_most_common(mod23_w12_train_sanity_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_w12_valid_seen_tp_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_w12_valid_unseen_tp_stc_obj_tokens)

Train Sanity
('tomato', 116)
('potato', 113)
('apple', 86)
('egg', 68)
('bread', 67)
('knife', 63)
('lettuce', 62)
('butterknife', 52)
('mug', 41)
('cup', 34)
('cloth', 31)
('pan', 27)
('spoon', 26)
('soapbar', 25)
('cellphone', 24)
('plate', 23)
('spatula', 20)
('pencil', 19)
('pot', 19)
('keychain', 18)
--------------------------------
Valid Seen
('tomato', 161)
('apple', 104)
('potato', 87)
('egg', 77)
('lettuce', 72)
('knife', 60)
('butterknife', 52)
('bread', 36)
('mug', 35)
('plate', 32)
('soapbar', 32)
('cup', 27)
('remotecontrol', 27)
('pan', 24)
('spraybottle', 24)
('ladle', 23)
('microwave', 18)
('desklamp', 16)
('bowl', 14)
('keychain', 14)
--------------------------------
Valid Unseen
('egg', 198)
('lettuce', 165)
('apple', 99)
('tomato', 86)
('bread', 78)
('butterknife', 64)
('mug', 58)
('potato', 55)
('knife', 38)
('cup', 37)
('floorlamp', 30)
('pan', 29)
('soapbar', 29)
('keychain', 25)
('desklamp', 23)
('fork', 21)
('spoon', 20)
('bowl', 17)
('plate', 17)
('pencil', 16)

In [56]:
# mod 23 wt12 STAGE CHANGE FN
print('Train Sanity')
print_most_common(mod23_w12_train_sanity_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_w12_valid_seen_fn_stc_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_w12_valid_unseen_fn_stc_obj_tokens)

Train Sanity
('egg', 33)
('apple', 26)
('potato', 19)
('lettuce', 18)
('cabinet', 13)
('tomato', 11)
('bread', 9)
('mug', 7)
('bowl', 5)
('pen', 5)
('pencil', 5)
('pot', 5)
('soapbar', 5)
('knife', 5)
('creditcard', 4)
('microwave', 4)
('cloth', 4)
('plate', 4)
('keychain', 3)
('spoon', 3)
--------------------------------
Valid Seen
('tomato', 27)
('lettuce', 24)
('apple', 23)
('egg', 23)
('potato', 14)
('bread', 9)
('winebottle', 8)
('toiletpaper', 8)
('knife', 6)
('mug', 6)
('butterknife', 5)
('plate', 5)
('sink', 4)
('spraybottle', 4)
('pen', 4)
('box', 4)
('fork', 4)
('plunger', 4)
('creditcard', 4)
('ladle', 3)
--------------------------------
Valid Unseen
('lettuce', 62)
('egg', 21)
('cabinet', 14)
('cup', 11)
('apple', 9)
('bowl', 9)
('soapbar', 7)
('mug', 6)
('plunger', 6)
('peppershaker', 5)
('watch', 5)
('baseballbat', 5)
('pencil', 5)
('spoon', 4)
('keychain', 4)
('potato', 4)
('cloth', 4)
('toiletpaper', 4)
('basketball', 3)
('soapbottle', 3)


In [57]:
# mod 23 wt12 VIS TP
print('Train Sanity')
print_most_common(mod23_w12_train_sanity_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_w12_valid_seen_tp_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_w12_valid_unseen_tp_vis_obj_tokens)

Train Sanity
('countertop', 659)
('sink', 364)
('tomato', 274)
('fridge', 236)
('potato', 236)
('butterknife', 160)
('lettuce', 154)
('knife', 151)
('bread', 149)
('faucet', 145)
('diningtable', 127)
('cup', 117)
('mug', 110)
('bowl', 101)
('apple', 101)
('desk', 95)
('microwave', 93)
('spoon', 84)
('fork', 67)
('soapbar', 65)
--------------------------------
Valid Seen
('countertop', 681)
('sink', 383)
('tomato', 325)
('fridge', 222)
('apple', 191)
('knife', 175)
('bread', 175)
('butterknife', 173)
('lettuce', 153)
('potato', 138)
('diningtable', 127)
('faucet', 123)
('microwave', 109)
('cup', 91)
('ladle', 87)
('bowl', 83)
('mug', 78)
('soapbar', 74)
('plate', 67)
('fork', 64)
--------------------------------
Valid Unseen
('countertop', 835)
('sink', 461)
('faucet', 235)
('tomato', 221)
('butterknife', 203)
('lettuce', 165)
('microwave', 153)
('bread', 150)
('apple', 144)
('potato', 127)
('mug', 123)
('fridge', 119)
('desk', 119)
('knife', 111)
('egg', 89)
('bowl', 87)
('desklamp', 7

In [58]:
# mod 23 wt12 VIS FN
print('Train Sanity')
print_most_common(mod23_w12_train_sanity_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Seen')
print_most_common(mod23_w12_valid_seen_fn_vis_obj_tokens)
print('--------------------------------')
print('Valid Unseen')
print_most_common(mod23_w12_valid_unseen_fn_vis_obj_tokens)

Train Sanity
('drawer', 1476)
('cabinet', 1180)
('stoveburner', 360)
('sink', 318)
('chair', 317)
('spatula', 218)
('stoveknob', 217)
('soapbottle', 214)
('fork', 212)
('mug', 201)
('bread', 187)
('faucet', 183)
('potato', 179)
('shelf', 171)
('knife', 163)
('spoon', 156)
('tomato', 153)
('apple', 149)
('saltshaker', 143)
('peppershaker', 143)
--------------------------------
Valid Seen
('cabinet', 1475)
('drawer', 1222)
('sink', 401)
('stoveburner', 296)
('spatula', 256)
('stoveknob', 241)
('chair', 239)
('faucet', 227)
('bread', 219)
('lettuce', 210)
('apple', 205)
('fork', 197)
('soapbottle', 195)
('tomato', 189)
('knife', 161)
('garbagecan', 156)
('pan', 154)
('spoon', 151)
('pot', 141)
('potato', 138)
--------------------------------
Valid Unseen
('cabinet', 887)
('drawer', 837)
('stoveburner', 300)
('sink', 286)
('statue', 276)
('bread', 270)
('shelf', 268)
('spatula', 264)
('lettuce', 263)
('cellphone', 255)
('apple', 242)
('pot', 235)
('knife', 229)
('butterknife', 226)
('tomat